### LUMO - Deep Convolutional Network in TensorFlow

#### Load dependencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import tensorflow as tf
from datetime import datetime
from dateutil import tz
from IPython import embed
import time
import socket

from pathlib import Path
from sklearn.metrics import confusion_matrix

np.random.seed(42)

%matplotlib inline
plt.style.use('ggplot')

C:\Users\adam\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### Add model-specific naming conventions

In [ ]:
#user input for filename of saved model, defaults to timestamp
name = input("String to add to model filename, defaults to time stamp if nothing entered.")  # Python 3
if name == "":
    name = time.time()
#user input to add note to plot
plot_note=input("Note you'd like to add to plot:")

#user input to load prev model
model_to_load=input("enter the model name to load - leave blank to start fresh")

#user input to load prev model
results_file_name=input("name of results file")


#### Set hyperparameters

In [ ]:
############################################################
# File setup                                               #
############################################################

#Set to true to report on hold out test set
FINALTEST=True

# Location of file containing CSV of training, test instances
myFileLocation='../datasets/big-mod.csv'

# Number of epochs between each printed update
checkin_frequency =  10

############################################################
# Hyperparameters                                          #
############################################################

batch_size = 50
kernel_size = 6
depth = 40
num_hidden = 50
reportingMetric_1="ACC" # options are "MAE", "MSE", "MAPE", "ACC", "ACC1", "ACC2"

reportingMetric_2="ACC2" # options are "MAE", "MSE", "MAPE", "ACC","ACC1", "ACC2"

reportingMetric_3="MAE" # options are "MAE", "MSE", "MAPE", "ACC","ACC1", "ACC2"

reportingMetric_4="" # options are "MAE", "MSE", "MAPE", "ACC","ACC1", "ACC2"

################Hyperparameters
sample_stride=18
input_height = 1
input_num_timestamps = 36
num_channels = 11
max_pool_kernel_size=6

speed_bucket_size = "0.1" #options are ".1" , ".5", "none_use_regression"
label_window_size = 30 # number of timestamps used to label the speed we will be predicting

loss_function = "cross_entropy" #options are: "L1_loss" , "cross_entropy" , "L2_loss"
optimizer_type = "gradient" #options are: "adam" , "rmsprop", "gradient"

activation_1 = "relu" #options are "relu" , "tanh" and "sigmoid" - used for depthwise_conv
activation_2 = "tanh" #options are "relu" , "tanh" and "sigmoid" - used for final FC layer

learning_rate = 0.0001
training_epochs = 500

#### Define functions for data processing and plotting

In [ ]:
def depthConvActivation(x,weights,biases):
    if activation_1=="relu":
        result=tf.nn.relu(tf.add(depthwise_conv2d(x, weights),biases)) 
        
    elif activation_1=="tanh":
        result=tf.nn.tanh(tf.add(depthwise_conv2d(x, weights),biases))
        
    elif activation_1=="sigmoid":
        result=tf.nn.sigmoid(tf.add(depthwise_conv2d(x, weights),biases))
    return result

def FCActivation(c_flat,f_weights_l1,f_biases_l1):
    if activation_2=="relu":
        result=tf.nn.relu(tf.add(tf.matmul(c_flat, f_weights_l1),f_biases_l1))
        
    elif activation_2=="tanh":
        result=tf.nn.tanh(tf.add(tf.matmul(c_flat, f_weights_l1),f_biases_l1))
        
    elif activation_2=="sigmoid":
        result=tf.nn.sigmoid(tf.add(tf.matmul(c_flat, f_weights_l1),f_biases_l1))
        
    return result
    
def load_results_file(results_file_name):
    my_file = Path("../results/"+results_file_name+".csv")
    if my_file.is_file():
        print("Found results file")
        prev_results=pd.read_csv(my_file,header=0)
        print(list(prev_results.columns.values))
        return prev_results
    else:
        print("no results file found - creating file")
        a=[["cnn",
            name,
            "plot_filename",
            myFileLocation,
            loss_function,
            optimizer_type,
            activation_1,
            activation_2,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,    
            training_epochs,
            0,
            0,
            0,
            batch_size,
            kernel_size,
            depth,
            num_hidden,
            max_pool_kernel_size,
            learning_rate,
            speed_bucket_size,
            loss_function,
            input_num_timestamps,
            checkin_frequency,
            plot_note,
            "none",
            results_file_name,
            num_channels,
            label_window_size,
            sample_stride
            ]]
        
        df=pd.DataFrame(a, columns=["model type",
                                    "model filename",
                                    "plot filename",
                                    "data filename",
                                    "loss function",
                                    "optimizer",
                                    "activation_1",
                                    "activation_2",
                                    "last_train_accuracy_1_bucket",
                                    "last_dev_accuracy_1_bucket",
                                    "last_train_accuracy_2_buckets",
                                    "last_dev_accuracy_2_buckets",
                                    "last_train_MSE",
                                    "last_dev_MSE",
                                    "last_train_MAE",
                                    "last_dev_MAE",
                                    "last_train_MAPE",
                                    "last_dev_MAPE",
                                    "epochs",
                                    "runtime",
                                    "dev accuracy",
                                    "train accuracy",
                                    "batch_size",
                                    "kernel_size",
                                    "depth",
                                    "num_hidden",
                                    "max_pool_kernel_size",
                                    "learning_rate",
                                    "speed_bucket_size",
                                    "loss_function",
                                    "input_num_timestamps",
                                    "checkin_frequency",
                                    "plot_note",
                                    "model_loaded",
                                    "results_file_name",
                                    "num_channels",
                                    "label_window_size",
                                    "sample_stride"])
        
        df.to_csv("../results/"+results_file_name+".csv",index=False )
        return df
    
def read_data(file_path):
    data = pd.read_csv(file_path,header = 0)
    return data

def feature_normalize(dataset):
    mu = np.mean(dataset,axis = 0)
    sigma = np.std(dataset,axis = 0)
    return (dataset - mu)/sigma
    
def plot_axis(ax, x, y, title):
    ax.plot(x, y)
    ax.set_title(title)
    ax.xaxis.set_visible(False)
    ax.set_ylim([min(y) - np.std(y), max(y) + np.std(y)])
    ax.set_xlim([min(x), max(x)])
    ax.grid(True)
    
def plot_activity(activity,data):
    fig, (ax0, ax1, ax2) = plt.subplots(nrows = 3, figsize = (15, 10), sharex = True)
    plot_axis(ax0, data['timestamp'], data['bounce'], 'bounce')
    plot_axis(ax1, data['timestamp'], data['braking'], 'braking')
    plot_axis(ax2, data['timestamp'], data['pelvic_tilt'], 'pelvic_tilt')
    plt.subplots_adjust(hspace=0.2)
    fig.suptitle(activity)
    plt.subplots_adjust(top=0.90)
    plt.show()
    
def windows(data, size):
    start = 0
    while start < data.count():
        yield int(start), int(start + size)
        start += sample_stride #(size / 2)

def segment_signal(data,window_size = input_num_timestamps):
    segments = np.empty((0,window_size,num_channels))
    labels = np.empty((0))
    for (start, end) in windows(data['timestamp'], window_size):
        #print(start)
        #print(end)
        aa = data["age"][start:end]
        bb = data["weight"][start:end]
        cc = data["height"][start:end]
        dd = data["gender"][start:end]
        a = data["bounce"][start:end]
        b = data["braking"][start:end]
        c = data["cadence"][start:end]
        d = data["ground_contact"][start:end]
        e = data["pelvic_drop"][start:end]
        f = data["pelvic_rotation"][start:end]
        g = data["pelvic_tilt"][start:end]
        if(end < data.shape[0] and
           len(data['timestamp'][start:end]) == window_size and
           data['activity_id'][start]==data['activity_id'][end]): 
            
            segments = np.vstack([segments,np.dstack([aa,bb,cc,dd,a,b,c,d,e,f,g])])
            
            start_labeling = np.int(np.floor(start+(end-start)/2) - np.floor(label_window_size/2))
            end_labeling = start_labeling + label_window_size
            
            if speed_bucket_size == "0.1":
                 labels = np.append(labels,np.around(np.mean(data["gps_speed_true"][start_labeling:end_labeling]),decimals=1)) 
                    # round to nearest decimal
            elif speed_bucket_size == "0.5":
                 labels = np.append(labels,np.around(2*np.mean(data["gps_speed_true"][start_labeling:end_labeling]),decimals=0)/2) 
            elif speed_bucket_size == "none_use_regression":
                 labels = np.append(labels,np.mean(data["gps_speed_true"][start_labeling:end_labeling]))
                    # round to nearest half unit
            
    num_labels = len(np.unique(labels))
    return segments, labels, num_labels

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)

def depthwise_conv2d(x, W):
    return tf.nn.depthwise_conv2d(x,W, [1, 1, 1, 1], padding='VALID')

def apply_depthwise_conv(x,kernel_size,num_channels,depth):
    weights = weight_variable([1, kernel_size, num_channels, depth])
    biases = bias_variable([depth * num_channels])
    result=depthConvActivation(x,weights,biases)
    
    return result
    
def apply_max_pool(x,kernel_size,stride_size):
    return tf.nn.max_pool(x, ksize=[1, 1, kernel_size, 1], 
                          strides=[1, 1, stride_size, 1], padding='VALID')

#### Read data from CSV into data frame

In [ ]:
# Load results of previously run models from file address specified in load_results_file()
past_results=load_results_file(results_file_name)
print(past_results)

# Read and normalize training/test instances
dataset = read_data(myFileLocation)

dataset['weight'] = feature_normalize(dataset['weight'])
dataset['height'] = feature_normalize(dataset['height'])
dataset['bounce'] = feature_normalize(dataset['bounce'])
dataset['gender'] = feature_normalize(dataset['gender'])
dataset['age'] = feature_normalize(dataset['age'])
dataset['braking'] = feature_normalize(dataset['braking'])
dataset['cadence'] = feature_normalize(dataset['cadence'])
dataset['ground_contact'] = feature_normalize(dataset['ground_contact'])
dataset['pelvic_drop'] = feature_normalize(dataset['pelvic_drop'])
dataset['pelvic_rotation'] = feature_normalize(dataset['pelvic_rotation'])
dataset['pelvic_tilt'] = feature_normalize(dataset['pelvic_tilt'])

#### Reshape individual data points into windowed training instances

In [ ]:
segments, labels, num_labels = segment_signal(dataset)

labels_as_numbers = np.asarray(np.unique(labels), dtype = np.float32)
labels_as_numbers = np.reshape(labels_as_numbers, (len(labels_as_numbers), 1))
labels_as_one_hots = np.asarray(pd.get_dummies(labels), dtype = np.int8)

reshaped_segments = segments.reshape(len(segments), 1,input_num_timestamps, num_channels)

#### Split data into train (90%) and dev (10%)

In [ ]:
train_test_split = np.random.rand(len(reshaped_segments)) < 0.90
print(train_test_split)

train_x = reshaped_segments[train_test_split]
train_y = labels_as_one_hots[train_test_split]

test_dev_x = reshaped_segments[~train_test_split]
test_dev_y = labels_as_one_hots[~train_test_split]

test_dev_split=np.random.rand(len(test_dev_x)) <.5

# In our class, this should be called dev, not test, but use "test" for consistency with other code-bases
test_x=test_dev_x[test_dev_split]
test_y=test_dev_y[test_dev_split]


hold_out_test_x=test_dev_x[~test_dev_split]
hold_out_test_y=test_dev_y[~test_dev_split]

print("train x shape:",train_x.shape)
print("test x shape:", test_x.shape)

total_batches = train_x.shape[0] // batch_size

#### Create TensorFlow objects and model architecture

In [ ]:
X = tf.placeholder(tf.float32, shape=[None,input_height,input_num_timestamps,num_channels])
Y = tf.placeholder(tf.float32, shape=[None,num_labels])

##Layer 1 - Depthwise Convolution
c = apply_depthwise_conv(X,kernel_size,num_channels,depth)

## Max Pool
p = apply_max_pool(c,max_pool_kernel_size,2)

##Layer 2 - Depthwise Convolution
c = apply_depthwise_conv(p,6,depth*num_channels,depth//10)

##Flatten for FC layers 
shape = c.get_shape().as_list()
print("SHAPE:",shape)

c_flat = tf.reshape(c, [-1, shape[1] * shape[2] * shape[3]])

##Initiate hidden layers weights/biases based on hyperparameter entered
f_weights_l1 = weight_variable([shape[1] * shape[2] * depth * num_channels * (depth//10), num_hidden])
f_biases_l1 = bias_variable([num_hidden])

## run num_hidden FC layers 
f = FCActivation(c_flat,f_weights_l1,f_biases_l1)

##Final Output weights/biases
out_weights = weight_variable([num_hidden, num_labels])
out_biases = bias_variable([num_labels])

## Final Layer - Softmax - unlress flagged for regression
if speed_bucket_size == "none_use_regression":
    y_ = tf.matmul(f, out_weights) + out_biases
elif speed_bucket_size != "none_use_regression":
    y_ = tf.nn.softmax(tf.matmul(f, out_weights) + out_biases)

    
numeric_Y = Y @ labels_as_numbers         #actual values
numeric_y_ = y_ @ labels_as_numbers       #predicted values

y_true_final = [] # placeholder list for (m/s) values of actual y
y_pred_final = [] # placeholder list for (m/s) values of predicted y

# Specifying loss function from flag hyperparameter
if loss_function == "cross_entropy":
    loss = -tf.reduce_sum(Y * tf.log(y_))
elif loss_function == "L1_loss":
    loss = tf.reduce_sum(np.abs(numeric_Y - numeric_y_))
elif loss_function == "L2_loss":
        loss = tf.sqrt(tf.reduce_sum((numeric_Y - numeric_y_) * (numeric_Y - numeric_y_)))

# Specifying optimizer from flag hyperparameter
if optimizer_type=="gradient":
    optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(loss)
elif optimizer_type=="adam":
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)
elif optimizer_type=="rmsprop":
    optimizer = tf.train.RMSPropOptimizer(learning_rate = learning_rate).minimize(loss)

#Creating prediction and error variables
correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(Y,1))
prediction_within_1_bucket = np.abs(tf.argmax(y_,1) - tf.argmax(Y,1)) <= 1
prediction_within_2_bucket = np.abs(tf.argmax(y_,1) - tf.argmax(Y,1)) <= 2

accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
accuracy_1_bucket = tf.reduce_mean(tf.cast(prediction_within_1_bucket, tf.float32))
accuracy_2_buckets = tf.reduce_mean(tf.cast(prediction_within_2_bucket, tf.float32))
MSE = tf.reduce_mean((numeric_Y - numeric_y_)*(numeric_Y - numeric_y_))
MAE = tf.reduce_mean(np.abs((numeric_Y - numeric_y_)))
MAPE = tf.reduce_mean(np.abs((numeric_Y - numeric_y_)))

# Instantiate reporting variables and lists to store their histories in:
cost_history = np.empty(shape=[1],dtype=float)

# Accuracy (only for display in classification tasks):
last_train_accuracy = 0.0
last_dev_accuracy = 0.0
dev_accuracy_history=np.empty(shape=[1],dtype=float)
train_accuracy_history=np.empty(shape=[1],dtype=float)

# Accuracy within 1 bucket (only for display in classification tasks):
last_train_accuracy_1_bucket = 0.0
last_dev_accuracy_1_bucket = 0.0
dev_accuracy_1_bucket_history=np.empty(shape=[1],dtype=float)
train_accuracy_1_bucket_history=np.empty(shape=[1],dtype=float)

# Accuracy withing 2 buckets (only for display in classification tasks):
last_train_accuracy_2_buckets = 0.0
last_dev_accuracy_2_buckets = 0.0
dev_accuracy_2_buckets_history = np.empty(shape=[1],dtype=float)
train_accuracy_2_buckets_history = np.empty(shape=[1],dtype=float)

# MSE (Mean Squared Error): 
last_train_MSE=0.0
last_dev_MSE=0.0
dev_MSE_history=np.empty(shape=[1],dtype=float)
train_MSE_history=np.empty(shape=[1],dtype=float)

# MAE (Mean Absolute Error): 
last_train_MAE=0.0
last_dev_MAE=0.0
dev_MAE_history=np.empty(shape=[1],dtype=float)
train_MAE_history=np.empty(shape=[1],dtype=float)

# MAPE (Mean Absolute Percentage Error): 
last_train_MAPE=0.0
last_dev_MAPE=0.0
dev_MAPE_history=np.empty(shape=[1],dtype=float)
train_MAPE_history=np.empty(shape=[1],dtype=float)


#### Run model and print ongoing results

In [ ]:
#create instance of saver to save model 
saver = tf.train.Saver()

start_time=time.time()

with tf.Session() as session:
    tf.global_variables_initializer().run()
    if model_to_load!="":
        saver.restore(session, "../models/"+model_to_load+".ckpt")
    counter =  0
    cost_at_epoch=[]
    last_checkin_time=time.time()
    
    
    for epoch in range(training_epochs):
        for b in range(total_batches):    
            offset = (b * batch_size) % (train_y.shape[0] - batch_size)
            batch_x = train_x[offset:(offset + batch_size), :, :, :]
            batch_y = train_y[offset:(offset + batch_size), :]
            _, c = session.run([optimizer, loss],feed_dict={X: batch_x, Y : batch_y})
            cost_history = np.append(cost_history,c)
        cost_at_epoch.append(c)    


        counter += 1

        if (counter % checkin_frequency == 0):

            #Updated lists of reporting metrics:

            #Accuracy (only for display in classification tasks):
            last_train_accuracy = session.run(accuracy, feed_dict={X: train_x, Y: train_y})
            last_dev_accuracy=session.run(accuracy, feed_dict={X: test_x, Y: test_y})
            train_accuracy_history=np.append(train_accuracy_history,last_train_accuracy)
            dev_accuracy_history=np.append(dev_accuracy_history,last_dev_accuracy)

            #Accuracy within 1 bucket (only for display in classification tasks):
            last_train_accuracy_1_bucket = session.run(accuracy_1_bucket, feed_dict={X: train_x, Y: train_y})
            last_dev_accuracy_1_bucket = session.run(accuracy_1_bucket, feed_dict={X: test_x, Y: test_y})
            train_accuracy_1_bucket_history = np.append(train_accuracy_1_bucket_history, last_train_accuracy_1_bucket)
            dev_accuracy_1_bucket_history = np.append(dev_accuracy_1_bucket_history,last_dev_accuracy_1_bucket)

            #Accuracy within 2 buckets (only for display in classification tasks):
            last_train_accuracy_2_buckets = session.run(accuracy_2_buckets, feed_dict={X: train_x, Y: train_y})
            last_dev_accuracy_2_buckets = session.run(accuracy_2_buckets, feed_dict={X: test_x, Y: test_y})
            train_accuracy_2_buckets_history = np.append(train_accuracy_2_buckets_history,last_train_accuracy_2_buckets)
            dev_accuracy_2_buckets_history = np.append(dev_accuracy_2_buckets_history,last_dev_accuracy_2_buckets)

            #MSE (Mean Squared Error):
            last_train_MSE = session.run(MSE, feed_dict={X: train_x, Y: train_y})
            last_dev_MSE=session.run(MSE, feed_dict={X: test_x, Y: test_y})
            train_MSE_history=np.append(train_MSE_history,last_train_MSE)
            dev_MSE_history=np.append(dev_MSE_history,last_dev_MSE)

            #MAE (Mean Absolute Error):
            last_train_MAE = session.run(MAE, feed_dict={X: train_x, Y: train_y})
            last_dev_MAE=session.run(MAE, feed_dict={X: test_x, Y: test_y})
            train_MAE_history=np.append(train_MAE_history,last_train_MAE)
            dev_MAE_history=np.append(dev_MAE_history,last_dev_MAE)

            #MAPE (Mean Absolute Percentage Error):
            last_train_MAPE = session.run(MAPE, feed_dict={X: train_x, Y: train_y})
            last_dev_MAPE = session.run(MAPE, feed_dict={X: test_x, Y: test_y})
            train_MAPE_history=np.append(train_MAPE_history,last_train_MAPE)
            dev_MAPE_history=np.append(dev_MAPE_history,last_dev_MAPE)

            print("Epoch: ",epoch,"*",checkin_frequency,"\n"+"Training Loss: ",c)
            print("Testing Accuracy:",last_dev_accuracy,"\n","Training Accuracy: ", last_train_accuracy)
            print("dev MAE:", last_dev_MAE)
            print("dev MSE:", last_dev_MSE)


            #print("Current Time:"+str(time.time()))
            print("Time cost:"+str(round(time.time()-last_checkin_time,1)))
            last_checkin_time=time.time()


            if counter > 1:
                # Save the variables to disk.
                save_path = saver.save(session, "../models/"+name+".ckpt")
                print("Model saved in path: %s" % save_path)
                print("\n")

    print("Testing Accuracy:", session.run(accuracy, feed_dict={X: test_x, Y: test_y}))  
   
    y_true_final = session.run(numeric_Y, feed_dict={X: test_x, Y: test_y})
    y_pred_final = session.run(numeric_y_, feed_dict={X: test_x, Y: test_y})
    
    if FINALTEST==True:
        y_true_final_test = session.run(numeric_Y, feed_dict={X: hold_out_test_x, Y: hold_out_test_y})
        y_pred_final_test= session.run(numeric_y_, feed_dict={X: hold_out_test_x, Y: hold_out_test_y})
        
        hold_out_MAE=session.run(MAE, feed_dict={X: hold_out_test_x, Y: hold_out_test_y})
        hold_out_MSE=session.run(MSE, feed_dict={X: hold_out_test_x, Y: hold_out_test_y})
        hold_out_ACC=session.run(accuracy, feed_dict={X: hold_out_test_x, Y: hold_out_test_y})
        hold_out_ACC1=session.run(accuracy_1_bucket, feed_dict={X: hold_out_test_x, Y: hold_out_test_y})
        hold_out_ACC2=session.run(accuracy_2_buckets, feed_dict={X: hold_out_test_x, Y: hold_out_test_y})

## Save detailed results 

In [ ]:
# Record pred vs actual in .csv

# Record data in a an .csv
y_trueVy_pred = np.hstack([y_true_final,y_pred_final])
df_y_trueVy_pred = pd.DataFrame(y_trueVy_pred)
filepath_predictions = "../results/"+results_file_name+"_Predictions_"+name+".csv"
df_y_trueVy_pred.to_csv(filepath_predictions, header = ["y_true_final", "y_pred_final"], index=False)


#Record accuracy metrics

# Save results to a .csv
df_devAccuracy = pd.DataFrame(np.transpose(np.vstack([dev_accuracy_history,
                                                      dev_accuracy_1_bucket_history,
                                                      dev_accuracy_2_buckets_history,
                                                      dev_MSE_history,
                                                      dev_MAE_history,
                                                      dev_MAPE_history
                                                      
                                                     ])))
filepath_acc = "../results/"+results_file_name+"_AccuracyPerEpoch_"+name+".csv"
df_devAccuracy.to_csv(filepath_acc, header = ["dev_accuracy_history",
                                              "dev_accuracy_1_bucket_history",
                                              "dev_accuracy_2_buckets_history",
                                              "dev_MSE_history",
                                              "dev_MAE_history",
                                              "dev_MAPE_history"
                                             ], index=False)


## Confusion Matrix

In [ ]:
print(y_true_final.shape)

plt.scatter(y_true_final, y_pred_final, s=10, alpha=0.5)
plt.show()


plot_filename="CM"+str(time.time())+"User-"+str(socket.gethostname())

plt.savefig("../CM/"+plot_filename+".png",bbox_inches='tight')


plt.scatter(y_true_final_test, y_pred_final_test, s=10, alpha=0.5)
plt.show()


plot_filename="CM"+str(time.time())+"User-"+str(socket.gethostname())

plt.savefig("../CM/"+plot_filename+".png",bbox_inches='tight')

print("hold_out_MAE:",hold_out_MAE)
print("hold_out_MSE:",hold_out_MSE)
print("hold_out_ACC:",hold_out_ACC)
print("hold_out_ACC1:",hold_out_ACC1)
print("hold_out_ACC2:",hold_out_ACC2)

#### Plot model results

In [ ]:
end_time=time.time()
model_time=end_time-start_time
time_per_epoch=model_time/training_epochs
from matplotlib.patches import Rectangle

def createLines(metricType):
    if (metricType=="MAE"):
        
        test=np.squeeze(dev_MAE_history[1:])
        train=np.squeeze(train_MAE_history[1:])
        dev=plt2.plot(test,'#6f0000',label='Mean Abs Error',linewidth=1.5)
        train=plt2.plot(train,'#ff4646',label='Mean Abs Error',linewidth=.8)
        return dev,train
        
    if (metricType=="MSE"):
        
        test=np.squeeze(dev_MSE_history[1:])
        train=np.squeeze(train_MSE_history[1:])
        dev=plt2.plot(test,'#315047',label='MSE',linewidth=1.5)
        train=plt2.plot(train,'#38efab',label='MSE',linewidth=.8)
        return dev,train
        
    if (metricType=="ACC"):
        
        test=np.squeeze(trainAccuracy[1:])
        train=np.squeeze(devAccuracy[1:])
        dev=plt2.plot(test,'#011f4b',label='Class. Accuracy',linewidth=1.5)
        train=plt2.plot(train,'#380356',label='Class. Accuracy',linewidth=.8)
        return dev,train
    
    if (metricType=="ACC1"):
        
        test=np.squeeze(dev_accuracy_1_bucket_history[1:])
        train=np.squeeze(train_accuracy_1_bucket_history[1:])
        dev=plt2.plot(test,'#005b96',label='ACC1',linewidth=1.5)
        train=plt2.plot(train,'#6b0078',label='ACC1',linewidth=.8)
        return dev,train
    
    if (metricType=="ACC2"):
        test=np.squeeze(dev_accuracy_2_buckets_history[1:])
        train=np.squeeze(train_accuracy_2_buckets_history[1:])
        dev=plt2.plot(test,'#6497b1',label='.2 Buffer Accuracy',linewidth=1.5)
        train=plt2.plot(train,'#b000b2',label='.2 Buffer Accuracy',linewidth=.8)
        return dev, train
    

trainAccuracy=np.squeeze(train_accuracy_history[1:])
devAccuracy =np.squeeze(dev_accuracy_history[1:])

fig, ax1 = plt.subplots()
lines=[]
lineLabels=[]
plt2=ax1.twinx()

if reportingMetric_1 != "":
    myLines = createLines(reportingMetric_1)
    lines += myLines[0]
    lines += myLines[1]
    lineLabels += [reportingMetric_1+'Dev']
    lineLabels += [reportingMetric_1+'Train']

if reportingMetric_2 != "":
    myLines = createLines(reportingMetric_2)
    lines += myLines[0]
    lines += myLines[1]
    lineLabels += [reportingMetric_2+'Dev']
    lineLabels += [reportingMetric_2+'Train']

if reportingMetric_3 != "":
    myLines = createLines(reportingMetric_3)
    lines += myLines[0]
    lines += myLines[1]
    lineLabels += [reportingMetric_3+'Dev']
    lineLabels += [reportingMetric_3+'Train']
    
if reportingMetric_4 != "":
    myLines = createLines(reportingMetric_4)
    lines += myLines[0]
    lines += myLines[1]
    lineLabels += [reportingMetric_4+'Dev']
    lineLabels += [reportingMetric_4+'Train']
    
print(lineLabels)  
#lines += ax1.plot(devAccuracy,'#a93226', label='Dev Accuracy', linewidth=1)

#lines += plt2.plot(mae_test,'#a22776',label='MAE',linewidth=1)
#lines += plt2.plot(mae_train,'#b33334',label='MAE',linewidth=1)
#plt2=ax1.twinx()

#plt2.plot(np.squeeze(test_accuracy_history),"b-")

plt.ylim([0, 1]) 
plt.ylabel('Model Results')
plt.xlabel('epoch x'+str(checkin_frequency))
plt.title("Final Dev Accuracy =" + str(last_dev_accuracy))
extra = Rectangle((0, 0), 1, 1, fc="w", fill=False, edgecolor='none', linewidth=0)
plt.legend([extra,extra,extra,extra,extra,extra,extra,extra,extra,extra,extra,extra,extra,extra,extra,extra,extra,extra,extra,extra,extra,extra,extra,extra],(
                                                
                                                "MAE" +str(last_dev_MAE),
                                                "MSE" +str(last_dev_MSE),
                                                "MAPE" +str(last_dev_MAPE),
                                                "ACC1" +str(last_dev_accuracy_1_bucket),
                                                "ACC2" +str(last_dev_accuracy_2_buckets),
                                                "Label win size:" +str(label_window_size),
                                                "Optimization:" + optimizer_type,
                                                "activation1:" + activation_1,
                                                "activation2:"+ activation_2,
                                                "loss: " + loss_function,
                                                "learning rate: " + str(learning_rate),
                                                "batch_size: " + str(batch_size),
                                                "kernel_size: " + str(kernel_size),
                                                "depth: "+ str(depth),
                                                "layers : "+ str(num_hidden),
                                                "max pool kernel size: "+str(max_pool_kernel_size),
                                                "speed_bucket_size: " + speed_bucket_size,
                                                "learning rate: "+str(learning_rate),
                                                "epochs: "+str(training_epochs),
                                                "input_num_timestamps: " + str(input_num_timestamps),
                                                "num_labels: " + str(num_labels),
                                                "num_channels:" + str(num_channels),
                                                "note:" + plot_note),
          
                                                bbox_to_anchor=(1.2, 1),
                                                loc=2,
                                                borderaxespad=0.)
from matplotlib.legend import Legend

leg = Legend(ax1, lines[0:], lineLabels[0:],bbox_to_anchor=(1,-.1,0,0),
             loc=1, frameon=True)

ax1.add_artist(leg);

plot_filename=str(time.time())+"User-"+str(socket.gethostname())

plt.savefig("../plots/"+plot_filename+".png",bbox_inches='tight')

a=[["cnn",
    name,
    plot_filename,
    myFileLocation,
    loss_function,
    optimizer_type,
    activation_1,
    activation_2,
    last_train_accuracy_1_bucket,
    last_dev_accuracy_1_bucket,
    last_train_accuracy_2_buckets,
    last_dev_accuracy_2_buckets,
    last_train_MSE,
    last_dev_MSE,
    last_train_MAE,
    last_dev_MAE,
    last_train_MAPE,
    last_dev_MAPE,    
    training_epochs,
    model_time,
    last_dev_accuracy,
    last_train_accuracy,
    batch_size,
    kernel_size,
    depth,
    num_hidden,
    max_pool_kernel_size,
    learning_rate,
    speed_bucket_size,
    loss_function,
    input_num_timestamps,
    checkin_frequency,
    plot_note,
    model_to_load,
    results_file_name,
    num_channels,
    label_window_size,
    sample_stride
    ]]
        
df=pd.DataFrame(a, columns=["model type",
                            "model filename",
                            "plot filename",
                            "data filename",
                            "loss function",
                            "optimizer",
                            "activation_1",
                            "activation_2",
                            "last_train_accuracy_1_bucket",
                            "last_dev_accuracy_1_bucket",
                            "last_train_accuracy_2_buckets",
                            "last_dev_accuracy_2_buckets",
                            "last_train_MSE",
                            "last_dev_MSE",
                            "last_train_MAE",
                            "last_dev_MAE",
                            "last_train_MAPE",
                            "last_dev_MAPE",
                            "epochs",
                            "runtime",
                            "dev accuracy",
                            "train accuracy",
                            "batch_size",
                            "kernel_size",
                            "depth",
                            "num_hidden",
                            "max_pool_kernel_size",
                            "learning_rate",
                            "speed_bucket_size",
                            "loss_function",
                            "input_num_timestamps",
                            "checkin_frequency",
                            "plot_note",
                            "model_loaded",
                            "results_file_name",
                            "num_channels",
                            "label_window_size",
                            "sample_stride"])


past_results=pd.concat([past_results,df])
past_results.to_csv("../results/"+results_file_name+".csv",index=False )
fig.tight_layout(pad=1)
plt.show()





    
    

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())